<a href="https://colab.research.google.com/github/joaomarcuslf2/data-science-studies/blob/main/imersao_alura_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

fonte = "https://github.com/alura-cursos/imersao-dados-2-2020/blob/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv?raw=true"
df = pd.read_csv(fonte)
subjects = ["NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]

Desafio 01: Criar uma função que facilite plotar um gráfico

In [ ]:
def create_plot(data, x, y, title = "", figsize = (10, 6), order = []):
  plt.figure(figsize=figsize)
  if (len(order) > 1):
    sns.boxplot(x=x, y=y, data=data, order=order)
  else:
    sns.boxplot(x=x, y=y, data=data)
  plt.title(title)

def plot_with_x_q006( y, title = "", order = []):
  return create_plot(df, "Q006", y, title, (10, 6), order)


In [ ]:
plot_with_x_q006(
    "NU_NOTA_MT", 
    "Boxplot das notas de matemática pela renda",
  )

In [ ]:
df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_MT", 
    "Boxplot das notas de matemática pela renda ordenada",
    df_by_salary
  )

In [ ]:
df["NU_NOTA_TOTAL"] = df[subjects].sum(axis=1)

df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_TOTAL", 
    "Boxplot das notas somadas pela renda",
    df_by_salary
  )

In [ ]:
df["NU_NOTA_MEDIA"] = df[subjects].mean(axis=1)

df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_MEDIA", 
    "Boxplot das notas somadas pela renda",
    df_by_salary
  )

In [ ]:
sns.displot(df, x="NU_NOTA_TOTAL")

In [ ]:
sns.displot(df, x="NU_NOTA_MEDIA")

In [ ]:
subjects.append("NU_NOTA_TOTAL")
subjects.append("NU_NOTA_MEDIA")

df.query("NU_NOTA_TOTAL == 0")[subjects]